In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch.nn import Linear, ReLU, Softmax
from torch.utils.data import Dataset, DataLoader
from load_data import load_normalized_data
from torch.nn.functional import relu, softmax
from tqdm import tqdm_notebook, tqdm
from IPython.display import clear_output

# Model Creation

In [17]:
class BasicMLP(torch.nn.Module):
    def __init__(self, input_dimensions, output_dimensions):
        super(BasicMLP, self).__init__()
        # Params: [0] = Input size to that layer. [1] = Number of nodes in that layer.
        self.input_layer = Linear(input_dimensions, 50)
        self.hidden_layer1 = Linear(50, 100)
        self.hidden_layer2 = Linear(100, 200)
        self.output_layer = Linear(200, output_dimensions)
        self.relu = ReLU()
        
    def forward(self, input_data):
        input_layer_output = self.relu(self.input_layer(input_data))
        hidden_layer_output1 = self.relu(self.hidden_layer1(input_layer_output))
        hidden_layer_output2 = self.relu(self.hidden_layer2(hidden_layer_output1))
        output_layer_output = self.output_layer(hidden_layer_output2)
        
        return output_layer_output

# Data Loader

In [18]:
class DenverDataset(Dataset):
    """Denver Hourly Weather Dataset."""

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __len__(self):
        return self.X_data.shape[0]

    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx][0]

# Load Data

In [19]:
X_train, X_test, y_train, y_test, num_classes = load_normalized_data("data/denver_data.csv", "weather_description_Denver")

### Convert to torch objects.

In [20]:
X_train = torch.tensor(X_train, requires_grad=True, dtype=torch.float).cuda()
y_train = torch.tensor(y_train, dtype=torch.long).cuda()
X_test = torch.tensor(X_test, requires_grad=True, dtype=torch.float).cuda()
y_test = torch.tensor(y_test,  dtype=torch.long).cuda()

### Create Dataset

In [21]:
batch_size = 400
training_set = DataLoader(DenverDataset(X_train, y_train), batch_size=batch_size, shuffle=True)

# Train Model

### Define Hyperparameters

In [22]:
def accuracy(outputs, targets):
    indices = torch.argmax(softmax(outputs, dim=1), 1)
    batch_size = indices.shape[0]

    num_correct = targets.eq(indices).sum()
    accuracy = (num_correct.item() / batch_size) * 100
    
    return accuracy

In [25]:
epochs = 1
model = BasicMLP(X_train.shape[1], num_classes).cuda() 
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

### Training Loop

In [ ]:
loop = tqdm_notebook(range(epochs))
for epoch in tqdm_notebook(range(epochs)):
    running_loss = 0.0
    
    for i, data in enumerate(training_set):
        
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
    
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if not i % 20:
            loop.set_description("Accuracy: {}%".format(accuracy(outputs, labels)))


print('Finished Training')


In [ ]:
# print('Final Loss: ', running_loss / )
#             loop.refresh()
        # print statistics
#         running_loss += loss.item()
#         if i % 20 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0